# 1 - Sequence to Sequence Learning with Neural Networks

+ In this series we'll be building a machine learning model to go from once sequence to another, using PyTorch and torchtext.
+ This will be done on German to English translations with [Multi30k dataset*](https://github.com/multi30k/dataset).

+ spaCy to assist in the tokenization of the data

  ```python
  python -m spacy download en_core_web_sm
  python -m spacy download de_core_news_sm
  ```

+ Reference: [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215) paper

+ *Multi30k is a dataset with ~30,000 parallel English, German and French sentences, each with ~12 words per sentence. 



# Introduction
+ The hidden state $(h_{i})$ as a vector representation of the sentence
+ The context vector $(z)$ as an abstract representation of the entire input sentence

+ **Decoder**, one per time-step.

  In the decoder, we need to go from the hidden state to an actual word, therefore at each time-step we use $s_t$ to predict (by passing it through a `Linear` layer, shown in purple) what we think is the next word in the sequence, $\hat{y}_t$. 

  $$\hat{y}_t = f(s_t)$$

  The words in the decoder are always generated one after another, with one per time-step. 

+ **Teacher Forcing**: ground truth + predicted word    
    
  We always use `<sos>` for the first input to the decoder, $y_1$, but for subsequent inputs, $y_{t>1}$, we will sometimes use the actual, ground truth next word in the sequence, $y_t$ and sometimes use the word predicted by our decoder, $\hat{y}_{t-1}$. This is called *teacher forcing*, see a bit more info about it [Teacher Forcing](https://machinelearningmastery.com/teacher-forcing-for-recurrent-neural-networks/). 

+ **Know the length in advance**

  When training/testing our model, we always know how many words are in our target sentence, so we stop generating words once we hit that many. During inference it is common to keep generating words until the model outputs an `<eos>` token or after a certain amount of words have been generated.

+ **Calculate the loss**

  Once we have our predicted target sentence, $\hat{Y} = \{ \hat{y}_1, \hat{y}_2, ..., \hat{y}_T \}$, we compare it against our actual target sentence, $Y = \{ y_{1}, y_{2}, ..., y_{T} \}$, to calculate our loss. We then use this loss to update all of the parameters in our model.

    

In [11]:
!pip install spacy
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

     |████████████████████████████████| 12.0 MB 7.9 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
     |████████████████████████████████| 14.9 MB 7.9 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')


In [16]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time


SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

+ Load Spacy model
+ Tokenizer Functions
  + reverse the order of the input
  + normal order for the output

  In the paper we are implementing, they find it beneficial to reverse the order of the input which they believe "introduces many short term dependencies in the data that make the optimization problem much easier". We copy this by reversing the German sentence after it has been transformed into a list of tokens.


+ torchtext's Fields 
  
  torchtext's Fields handle how data should be processed.

  + appends the "start of sequence" and "end of sequence" tokens
  + converts all words to lowercase


In [12]:
# Load Spacy model
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

# tokenizer functions
# transformed sentence into a list of tokens
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    (SRC-source)
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    (TRG-target)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]


# torchtext's Field
SRC = Field(tokenize = tokenize_de, 
          init_token = '<sos>', 
          eos_token = '<eos>', 
          lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

# Download Multi30k dataset
train_data, valid_data, test_data = Multi30k.splits(
    exts = ('.de', '.en'), fields = (SRC, TRG))

In [14]:
# check the data by its length
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

# print out an single example, make sure the source sentence is reversed:
from pprint import pprint
pprint(vars(train_data.examples[0]))

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000
{'src': ['.',
         'büsche',
         'vieler',
         'nähe',
         'der',
         'in',
         'freien',
         'im',
         'sind',
         'männer',
         'weiße',
         'junge',
         'zwei'],
 'trg': ['two',
         'young',
         ',',
         'white',
         'males',
         'are',
         'outside',
         'near',
         'many',
         'bushes',
         '.']}


# Build Vocabulary
use Torchtext's Filed object to build vocabulary

+ Build the vocabulary for the source and target languages. 

  The vocabulary is used to associate each unique token with an index (an integer). The vocabularies of the source and target languages are distinct.

+ Frequency Condition and Unknow Token(`<unk>`)

  Using the `min_freq` argument, we only allow tokens that appear at least 2 times to appear in our vocabulary. Tokens that appear only once are converted into an `<unk>` (unknown) token.

+ Vocabulary only be built from training set

  It is important to note that our vocabulary should only be built from the training set and not the validation/test set. This prevents "information leakage" into our model, giving us artifically inflated validation/test scores.


In [15]:
# use Filed object to build vocabulary
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 7855
Unique tokens in target (en) vocabulary: 5893


# Build iterators for DataSet 

In NLP, when we get a batch of examples using an iterator we need to make sure that all of the source sentences are padded to the same length, the same with the target sentences.

We use a `BucketIterator` instead of the standard `Iterator` as it creates batches in such a way that it minimizes the amount of padding in both the source and target sentences. 

In [17]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

# Building the Seq2Seq Model

#### Encoder 

+ 2 layer LSTM 

  (The paper we are implementing uses a 4-layer LSTM, but in the interest of training time we cut this down to 2-layers.)

#### Decoder

+ Also be a 2-layer (4 in the paper) LSTM

+ Decoding single token per time-step

  The Decoder class does a single step of decoding. i.e. it ouputs single token per time-step. only decoding one token at a time, the input tokens will always have a sequence length of 1

+ Context vectors as fist input in Decoder

  the initial hidden and cell states to our decoder are our context vectors, which are the final hidden and cell states of our encoder from the same layer, i.e. $(s_0^l,c_0^l)=z^l=(h_T^l,c_T^l)$.


#### Seq2Seq


In [17]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell


class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell